<a href="https://colab.research.google.com/github/MehmetDenizKaya/Llama-3.1-Training/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
"""fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [4]:
import pandas as pd
import json
import os

# Define the CSV file path
csv_file_path = "sonllmdata.csv"
processed_data_path = "processed_training_data.jsonl"

# Bu sözlük aynı kalabilir, soru tiplerini ve karşılık gelen sütun isimlerini tanımlıyor.
example_questions = {
    "Bu bölümün fakülte adı nedir?": "fakülte adı",
    "Hangi fakültede yer alıyor?": "fakülte adı",
    "Bu veriler hangi yıla ait?": "yıl",
    "Son güncellenen yıl nedir?": "yıl",
    "Alan tercihi nedir?": "alan tercihi",
    "Bu bölümün alanı ne?": "alan tercihi",
    "Öğrenim türü nedir?": "öğrenim türü",
    "Bu bölüm ikinci öğretim mi?": "öğrenim türü",
    "Üniversite adı nedir?": "üniversite adı",
    "Hangi üniversitede bulunuyor?": "üniversite adı",
    "Departman adı nedir?": "üniversiteye ait departman adı",
    "Hangi bölümden bahsediliyor?": "üniversiteye ait departman adı",
    "Toplam kontenjan kaç?": "toplam kontenjan",
    "Kaç kişilik kontenjan var?": "toplam kontenjan",
    "Toplam yerleşen öğrenci sayısı kaç?": "toplam yerleşen kişi sayısı",
    "Kaç kişi yerleşmiş?": "toplam yerleşen kişi sayısı",
    "Son giren öğrencinin puanı nedir?": "son girenin puanı",
    "En düşük puanla giren kaç puan aldı?": "son girenin puanı",
    "Son giren öğrencinin sırası nedir?": "son girenin sıralaması",
    "En düşük sırayla giren kaçıncı oldu?": "son girenin sıralaması",
    "İlk giren öğrencinin puanı nedir?": "ilk girenin puanı",
    "En yüksek puanla giren kaç puan aldı?": "ilk girenin puanı",
    "İlk giren öğrencinin sırası nedir?": "ilk girenin sıralaması",
    "En yüksek sırayla giren kaçıncı oldu?": "ilk girenin sıralaması",
}

instruction_text = """
    Aşağıdaki üniversite ve bölüm bilgisine dayanarak verilen soruya kısa ve doğrudan cevap ver.
    Kesinlikle kendi yorumunu, ek bilgiyi veya tahminleri ekleme. Yeni veriler üretme. Sadece training dataset içindeki verileri kullan.
    Sadece sağlanan üniversite, bölüm ve diğer ilgili bilgiyi temel alarak cevap ver.
    Eğer istenen bilgi sağlanmadıysa veya mevcut değilse, 'Bu bilgiye sahip değiliz.' şeklinde cevap ver.
    Tüm cevapları Türkçe ver.
    """

alpaca_format = """
    Instruction: {instruction}

    Üniversite Adı: {universite_adi}
    Bölüm Adı: {universiteye_ait_departman_adi}
    ### Soru: {question}
    ### Cevap: {answer}{eos_token}""" # Tokenizer'dan EOS token'ı ekleyeceğiz

try:
    print(f"Loading data from {csv_file_path}...")
    df = pd.read_csv(csv_file_path, encoding="utf-8", delimiter=";", decimal=",")
    print(f"Successfully loaded {len(df)} rows.")

    # Data cleaning for negative numeric values (aynı kalabilir)
    print("Applying data cleaning...")
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            numeric_col = pd.to_numeric(df[col], errors='coerce')
            df.loc[numeric_col.notna() & (numeric_col < 0), col] = None
    print("Data cleaning applied.")

    # Process data and write directly to JSONL file
    print(f"Processing data and writing to {processed_data_path}...")
    processed_example_count = 0
    if 'tokenizer' not in globals():
         print("Tokenizer bulunamadı. Lütfen modeli yüklediğiniz hücreyi çalıştırın.")
    eos_token = tokenizer.eos_token if hasattr(tokenizer, 'eos_token') else ""
    if eos_token is None: eos_token = "" # None olabilir bazı tokenizerlarda

    if not df.empty:
        with open(processed_data_path, 'w', encoding='utf-8') as f:
            for idx, row in df.iterrows():
                for question_phrase, column_name in example_questions.items():
                    if column_name in df.columns:
                         value = row[column_name]
                         ex_answer = str(value) if pd.notna(value) else "Bu bilgiye sahip değiliz."
                         # Yeni alpaca_format'a göre text oluşturalım
                         formatted_text = alpaca_format.format(
                             instruction=instruction_text.strip(), # Güncellenmiş Instruction'ı kullan
                             universite_adi=row.get("üniversite adı", "Bilinmiyor"), # Üniversite adını al
                             universiteye_ait_departman_adi=row.get("üniversiteye ait departman adı", "Bilinmiyor"), # Bölüm adını al
                             question=question_phrase,
                             answer=ex_answer,
                             eos_token=eos_token # EOS token'ı ekle
                         )
                         example = {"text": formatted_text}
                         f.write(json.dumps(example, ensure_ascii=False) + '\n')
                         processed_example_count += 1

        print(f"Finished writing {processed_example_count} training examples to {processed_data_path}")

    else:
        print("DataFrame is empty after loading or filtering.")

except FileNotFoundError as fnf_error:
    raise RuntimeError(f"Veri yüklenemedi - Dosya bulunamadı: {fnf_error}")
except pd.errors.EmptyDataError:
    raise RuntimeError(f"Veri yüklenemedi - Dosya boş: {csv_file_path}")
except pd.errors.ParserError as pe:
    raise RuntimeError(f"Veri yüklenemedi - Dosya ayrıştırma hatası: {pe}")
except Exception as e:
    # Genel hata yakalamada, hatanın ne olduğunu yazdırmak faydalı olabilir
    print(f"An unexpected error occurred during data processing: {e}")
    # Traceback yazdırmak, hatanın tam olarak nerede olduğunu gösterir
    import traceback
    traceback.print_exc()
    raise RuntimeError(f"Veri yüklenemedi - Beklenmedik bir hata oluştu: {e}")

Loading data from sonllmdata.csv...
Successfully loaded 67624 rows.
Applying data cleaning...
Data cleaning applied.
Processing data and writing to processed_training_data.jsonl...
Finished writing 1622976 training examples to processed_training_data.jsonl


In [7]:
# @title Hazırlanan Train Verisini Dataset olarak kayıt etme
# burası trained dataseti kaydetme işi
from datasets import load_dataset
import os # Dosyanın varlığını kontrol etmek için

# Define the path where the processed training data is saved (JSONL format)
processed_data_path = "/content/processed_training_data.jsonl"

try:
    # --- 2. Kaydedilmiş dosyadan Hugging Face Dataset oluştur ---

    # Dosyanın varlığını ve boş olup olmadığını kontrol et
    if os.path.exists(processed_data_path) and os.path.getsize(processed_data_path) > 0:
        print(f"Loading dataset from {processed_data_path}...")
        # load_dataset kullanarak JSONL dosyasından dataset oluştur
        # 'json' tipi kullanılır ve data_files olarak dosya yolu verilir
        # split='train' ile eğitim split'ini belirliyoruz
        dataset = load_dataset('json', data_files=processed_data_path, split='train')

        print("Created Hugging Face Dataset:")
        print(dataset)
        # Example of the first few training examples
        # if len(dataset) > 0:
        #     print("\nFirst example text:")
        #     for i in range(min(5, len(dataset))):
        #         print(f"--- Example {i+1} ---")
        #         print(dataset[i]['text'])
    else:
         print(f"Processed data file not found or is empty at {processed_data_path}. Cannot create dataset.")


except FileNotFoundError as fnf_error:
     # Bu aslında load_dataset tarafından yakalanmalı, ama ek güvenlik
    print(f"Error: Processed data file not found at {processed_data_path}")
except Exception as e:
    raise RuntimeError(f"Dataset oluşturulurken bir hata oluştu: {e}")

# Bu bloğun sonunda, eğer dosya başarıyla oluşturulduysa 'dataset' değişkeni
# Hugging Face Dataset objesini içerecektir ve diğer kod bloklarında
# (örn. trainer oluştururken) kullanılabilir.

Loading dataset from /content/processed_training_data.jsonl...


Generating train split: 0 examples [00:00, ? examples/s]

Created Hugging Face Dataset:
Dataset({
    features: ['text'],
    num_rows: 1622976
})


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import torch # GPU bellek kontrolü için

# Daha önce tanımlanmış model ve tokenizer objeleri burada kullanılıyor varsayılmıştır.
# dataset objesi de 9.1 milyon örneği içeriyor varsayılmıştır.
# max_seq_length = 4096 (veya metin uzunluklarına göre ayarlı)
# is_bfloat16_supported() çalıştırılmış ve True/False değeri belirlenmiş

# Mevcut bellek durumunu göster (opsiyonel ama faydalı)
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved BEFORE training.")


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096, # Metin uzunluklarına göre ayarla, 14GB için zorlayıcı olabilir. 2048 veya 1024 deneyebilirsin eğer OOM hatası alırsan.
    dataset_num_proc = 4, # Veri yüklemeyi hızlandırmak için çekirdek sayısını artırabilirsin.
    packing = False, # Güvenli başlangıç için False
    args = TrainingArguments(
        per_device_train_batch_size = 2, # 14GB bellek için başlangıçta en güvenli değer
        gradient_accumulation_steps = 16, # Efektif batch size'ı artırmak için deneysel olarak ayarla (1x64=64)
        warmup_steps = 30, # max_steps sayısına göre ayarla. Örneğin max_steps 500 ise %10'u (50 adım) gibi.
        num_train_epochs = 2, # max_steps kullanmak için -1 yap
        max_steps = 100, # 30 dakikaya göre deneysel olarak ayarla. Önce küçük bir değerle başla.
        learning_rate = 2e-4, # Makul başlangıç değeri
        fp16 = not is_bfloat16_supported(), # Unsloth tarafından otomatik ayarlanır
        bf16 = is_bfloat16_supported(),     # Unsloth tarafından otomatik ayarlanır
        logging_steps = 10, # Logları sık görmek için
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear", # Yaygın ve iyi performans veren scheduler
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        # Diğer faydalı argümanlar (isteğe bağlı):
        # save_steps = 500, # Model ağırlıklarını belirli adımlarda kaydetmek için
        # save_total_limit = 2, # Sadece en son 2 checkpoint'i tutmak için
    ),
)

# Eğitimi başlat
trainer_stats = trainer.train()

# Eğitim sonrası bellek ve süre istatistiklerini göster (önceden vardı, tekrar ekledim)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# start_gpu_memory zaten kodun üst kısmında hesaplanmış olmalı
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)

print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.135 GB of memory reserved BEFORE training.


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1622976 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,622,976 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.409000


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [12]:
alpaca_format = """
    Instruction: {instruction}
    ### Soru: {input}
    ### Cevap: {output}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_format.format(
        instruction="Şimdi bana dataset üzerinden bildiklerine göre yardımcı olman lazım", # instruction
        input="Türkiyedeki en iyi 10 üniversiteyi söyler misin", # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 90, use_cache = False)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>\n    Instruction: Şimdi bana dataset üzerinden bildiklerine göre yardımcı olman lazım\n    ### Soru: Türkiyedeki en iyi 10 üniversiteyi söyler misin\n    ### Cevap: 1. Boğaziçi Üniversitesi, 2. İTÜ, 3. Bilkent Üniversitesi, 4. Orta Doğu Teknik Üniversitesi, 5. Hacettepe Üniversitesi, 6. İstanbul Üniversitesi, 7. Yeditepe Üniversitesi, 8. Koç Üniversitesi, 9']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [13]:
alpaca_format = """
    Instruction: {instruction}
    ### Soru: {input}
    ### Cevap: {output}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_format.format(
        instruction="Sen kullanıcılara yardımcı olan bir yapay zeka modelisin.", # instruction
        input="Türkiyedeki en iyi 10 üniversiteyi söyler misin", # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer)

<|begin_of_text|>
    Instruction: Sen kullanıcılara yardımcı olan bir yapay zeka modelisin.
    ### Soru: Türkiyedeki en iyi 10 üniversiteyi söyler misin
    ### Cevap: 1- Boğaziçi Üniversitesi
    ### Cevap: 2- Koç Üniversitesi
    ### Cevap: 3- Orta Doğu Teknik Üniversitesi
    ### Cevap: 4- İstanbul Teknik Üniversitesi
    ### Cevap: 5- Bilkent Üniversitesi
    ### Cevap: 6- Yıldız Teknik Üniversitesi
    ### Cevap: 7- İTÜ
    ### Cevap: 8- Marmara Üniversitesi
    ### Cevap: 9- Anadolu Üniversitesi
    ### Cevap: 10- Ege Üniversitesi
    ### Cevap: 11- Hacettepe Üniversitesi
    ### Cevap: 12- Dokuz Eylül Üniversitesi
    ### Cevap: 13- İzmir Yüksek Teknoloji Enstitüsü
    ### Cevap: 14- Gaziantep Üniversitesi
    ### Cevap: 15- Mersin Üniversitesi
    ### Cevap: 16- Uludağ Üniversitesi
    ### Cevap: 17- Trakya Üniversitesi
    ### Cevap: 18- Çukurova 

KeyboardInterrupt: 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
